In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
sys.path.append("examples/")

import logging
import argparse
import json
from tqdm import tqdm, trange
import csv

import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining 
from pytorch_pretrained_bert.optimization import BertAdam

from torch.utils.data import Dataset
import random

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
from run_chunk_lm_finetune import InputExample, random_word, InputFeatures

In [3]:
# args
gradient_accumulation_steps = 1
train_batch_size = 1
eval_file = "dataset/dev-v2.0.json"
max_seq_length=256
on_memory = True
bert_model = "model_chunk/pytorch_model.bin"

In [4]:
class BERTDataset(Dataset):
    def __init__(self, corpus_path, tokenizer, seq_len, encoding="utf-8", on_memory=True, answerable_flag=True):
        self.vocab = tokenizer.vocab
        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.on_memory = on_memory
        self.corpus_path = corpus_path
        self.encoding = encoding

        # for loading samples directly from file
        self.sample_counter = 0  # used to keep track of full epochs on file
        self.line_buffer = None  # keep second sentence of a pair in memory and use as first sentence in next pair

        # for loading samples in memory
        self.questions = []
        self.contexts = []
        self.examples = []

        # load samples into memory
        if on_memory:
            # DANITER: Load Squad data
            with open(corpus_path, 'r') as handle:
                jdata = json.load(handle)
                data = jdata['data']

            for i in tqdm(range(len(data)), "Loading Squad", total=len(data)):
                section = data[i]['paragraphs']
                for sec in section:
                    context = sec['context']
                    self.contexts.append(context)
                    qas = sec['qas']
                    for j in range(len(qas)):
                        question = qas[j]['question']
                        unanswerable = qas[j]['is_impossible']
                        self.questions.append(question)
                        if unanswerable and answerable_flag:
                            continue
                        if not unanswerable and not answerable_flag:
                            continue
                        self.examples.append((len(self.contexts)-1, len(self.questions)-1))

#             with open("../training_data_chunks.pkl", "rb") as handle:
#                 self.training_data_map = pickle.load(handle)


        # load samples later lazily from disk
        else:
            raise Exception("No supported")

    def __len__(self):
        # last line of doc won't be used, because there's no "nextSentence". Additionally, we start counting at 0.
        return len(self.examples)

    def __getitem__(self, item):
        cur_id = self.sample_counter
        self.sample_counter += 1
        if not self.on_memory:
            raise Exception("No supported")

        while True:
            t1, t2, target, is_next_label = self.get_example(item)

            # tokenize
            tokens_a = self.tokenizer.tokenize(t1)
            tokens_b = self.tokenizer.tokenize(t2)
            if len(tokens_a) + len(tokens_b) + 3 > self.seq_len :
                item += 1
            else:
                break

        # combine to one sample
        cur_example = InputExample(guid=cur_id, tokens_a=tokens_a, tokens_b=tokens_b, is_next=is_next_label, target=target)

        # transform sample to features
        cur_features = convert_example_to_features(cur_example, self.seq_len, self.tokenizer)

        cur_tensors = (torch.tensor(cur_features.input_ids),
                       torch.tensor(cur_features.input_mask),
                       torch.tensor(cur_features.segment_ids),
                       torch.tensor(cur_features.lm_label_ids),
                       torch.tensor(cur_features.is_next))

        return cur_tensors

    def get_example(self, index):
        """
        Get one sample from corpus consisting of two sentences. With prob. 50% these are two subsequent sentences
        from one doc. With 50% the second sentence will be a random one from another doc.
        :param index: int, index of sample.
        :return: (str, str, int), sentence 1, sentence 2, isNextSentence Label
        """
        t1, t2 = self.get_corpus_line(index)

        target = (None, None) # keep same shape
        # Daniter we do not do next sentence prediction

        assert len(t1) > 0
        assert len(t2) > 0
        return t1, t2, target, 1

    def get_corpus_line(self, item):
        """
        Get one sample from corpus consisting of a pair of two subsequent lines from the same doc.
        :param item: int, index of sample.
        :return: (str, str), two subsequent sentences from corpus
        """
        t1 = ""
        t2 = ""
        assert item < len(self.examples)
        if self.on_memory:
            # DANITER - get the context and question pair based on the example indexes
            context_idx, question_idx = self.examples[item]
            t1 = self.contexts[context_idx]
            t2 = self.questions[question_idx]

            # used later to avoid random nextSentence from same doc
            return t1, t2
        else:
            raise Exception("No supported")

In [5]:
def convert_example_to_features(example, max_seq_length, tokenizer):
    """
    Convert a raw sample (pair of sentences as tokenized strings) into a proper training sample with
    IDs, LM labels, input_mask, CLS and SEP tokens etc.
    :param example: InputExample, containing sentence input as strings and is_next label
    :param max_seq_length: int, maximum length of sequence.
    :param tokenizer: Tokenizer
    :return: InputFeatures, containing all inputs and labels of one sample as IDs (as used for model training)
    """
    tokens_a = example.tokens_a
    tokens_b = example.tokens_b
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    # _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

    t1_random, t1_label = random_word(tokens_a, tokenizer, question=False)
    t2_random, t2_label = random_word(tokens_b, tokenizer, question=False)
    # concatenate lm labels and account for CLS, SEP, SEP
    lm_label_ids = ([-1] + t1_label + [-1] + t2_label + [-1])

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0   0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambigiously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    assert len(t2_random) > 0
    for token in t2_random:
        tokens.append(token)
        segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)
    while len(lm_label_ids) < max_seq_length:
        lm_label_ids.append(-1)

    assert len(input_ids) == max_seq_length, len(input_ids)
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length


    features = InputFeatures(input_ids=input_ids,
                             input_mask=input_mask,
                             segment_ids=segment_ids,
                             lm_label_ids=lm_label_ids,
                             is_next=example.is_next)
    return features

In [6]:
device = torch.device("cuda" if torch.cuda.is_available()  else "cpu")
n_gpu = torch.cuda.device_count()

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if n_gpu > 0:
    torch.cuda.manual_seed_all(42)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

# Load eval_data
eval_dataset_answerable = BERTDataset(eval_file, tokenizer, seq_len=max_seq_length,
                            on_memory=on_memory, answerable_flag=True)
eval_dataset_unanswerable = BERTDataset(eval_file, tokenizer, seq_len=max_seq_length,
                           on_memory=on_memory, answerable_flag=False)

# Prepare model
model_state_dict = torch.load(bert_model, map_location='cpu') #TODO daniter: remove this map_location
## TODO daniter: check if bert model is being loaded correctly
model = BertForPreTraining.from_pretrained("bert-base-uncased", state_dict=model_state_dict)
model.to(device)


# Prepare optimizer
print("Checking the vocab size:", len(tokenizer.vocab))
# 768 is bert hidden size, 256 is GRU hidden size, 1 is the layers in the GRU

# eval loader
eval_sampler_ans = SequentialSampler(eval_dataset_answerable)
eval_dataloader_ans = DataLoader(eval_dataset_answerable, sampler=eval_sampler_ans,
                                 batch_size=train_batch_size)
eval_sampler_unans = SequentialSampler(eval_dataset_unanswerable)
eval_dataloader_unans = DataLoader(eval_dataset_unanswerable, sampler=eval_sampler_unans,
                                   batch_size=train_batch_size)


03/06/2019 14:52:03 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/daniter/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
Loading Squad: 100%|██████████| 35/35 [00:00<00:00, 3988.50it/s]
03/06/2019 14:52:04 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/06/2019 14:52:04 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /Users/daniter/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a

Checking the vocab size: 30522


In [21]:
target = 120 #50
with torch.no_grad():
    model.eval()

    eval_loss_ans = 0
    for batch_i, eval_batch in enumerate(eval_dataloader_unans):
        eval_batch = tuple(t.to(device) for t in eval_batch)
        input_ids, input_mask, segment_ids, lm_label_ids, is_next = eval_batch
        if batch_i != target:
            continue
        if batch_i == target:
            input_ids[0][130] = 103
            input_ids[0][131] = 103
            #input_ids[0][117] = 103
            print(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]))
        output, _ = model(input_ids, segment_ids, input_mask, None, None)
        if batch_i == target:
            break

['[CLS]', 'a', 'problem', 'is', 'regarded', 'as', 'inherently', 'difficult', 'if', 'its', 'solution', 'requires', 'significant', 'resources', ',', 'whatever', 'the', 'algorithm', 'used', '.', 'the', 'theory', 'formal', '##izes', 'this', 'intuition', ',', 'by', 'introducing', 'mathematical', 'models', 'of', 'computation', 'to', 'study', 'these', 'problems', 'and', 'quan', '##tify', '##ing', 'the', 'amount', 'of', 'resources', 'needed', 'to', 'solve', 'them', ',', 'such', 'as', 'time', 'and', 'storage', '.', 'other', 'complexity', 'measures', 'are', 'also', 'used', ',', 'such', 'as', 'the', 'amount', 'of', 'communication', '(', 'used', 'in', 'communication', 'complexity', ')', ',', 'the', 'number', 'of', 'gates', 'in', 'a', 'circuit', '(', 'used', 'in', 'circuit', 'complexity', ')', 'and', 'the', 'number', 'of', 'processors', '(', 'used', 'in', 'parallel', 'computing', ')', '.', 'one', 'of', 'the', 'roles', 'of', 'computational', 'complexity', 'theory', 'is', 'to', 'determine', 'the', 'p

In [22]:
print(tokenizer.convert_ids_to_tokens(np.argmax(output[0].data.numpy(), axis=1)))

['what', 'a', 'problem', 'is', 'regarded', 'as', 'inherently', 'difficult', 'if', 'its', 'solution', 'requires', 'significant', 'resources', ',', 'whatever', 'the', 'algorithm', 'used', '.', 'the', 'theory', 'formal', '##izes', 'this', 'intuition', ',', 'by', 'introducing', 'mathematical', 'models', 'of', 'computation', 'to', 'study', 'these', 'problems', 'and', 'quan', '##tify', '##ing', 'the', 'amount', 'of', 'resources', 'needed', 'to', 'solve', 'them', ',', 'such', 'as', 'time', 'and', 'storage', '.', 'other', 'complexity', 'measures', 'are', 'also', 'used', ',', 'such', 'as', 'the', 'amount', 'of', 'communication', '(', 'used', 'in', 'communication', 'complexity', ')', ',', 'the', 'number', 'of', 'gates', 'in', 'a', 'circuit', '(', 'used', 'in', 'circuit', 'complexity', ')', 'and', 'the', 'number', 'of', 'processors', '(', 'used', 'in', 'parallel', 'computing', ')', '.', 'one', 'of', 'the', 'roles', 'of', 'computational', 'complexity', 'theory', 'is', 'to', 'determine', 'the', 'pr

In [20]:
print(list(zip(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]), range(256))))


[('[CLS]', 0), ('a', 1), ('problem', 2), ('is', 3), ('regarded', 4), ('as', 5), ('inherently', 6), ('difficult', 7), ('if', 8), ('its', 9), ('solution', 10), ('requires', 11), ('significant', 12), ('resources', 13), (',', 14), ('whatever', 15), ('the', 16), ('algorithm', 17), ('used', 18), ('.', 19), ('the', 20), ('theory', 21), ('formal', 22), ('##izes', 23), ('this', 24), ('intuition', 25), (',', 26), ('by', 27), ('introducing', 28), ('mathematical', 29), ('models', 30), ('of', 31), ('computation', 32), ('to', 33), ('study', 34), ('these', 35), ('problems', 36), ('and', 37), ('quan', 38), ('##tify', 39), ('##ing', 40), ('the', 41), ('amount', 42), ('of', 43), ('resources', 44), ('needed', 45), ('to', 46), ('solve', 47), ('them', 48), (',', 49), ('such', 50), ('as', 51), ('time', 52), ('and', 53), ('storage', 54), ('.', 55), ('other', 56), ('complexity', 57), ('measures', 58), ('are', 59), ('also', 60), ('used', 61), (',', 62), ('such', 63), ('as', 64), ('the', 65), ('amount', 66), ('

In [11]:
from collections import Counter

In [26]:
c = Counter()
for i, o in enumerate(output[0][134]):
    c[i] = o
for x, val in c.most_common(25):
    print(tokenizer.convert_ids_to_tokens([x]), val)

['complexity'] tensor(9.9284)
['the'] tensor(9.8887)
['limits'] tensor(9.8850)
['constraints'] tensor(9.7926)
['problems'] tensor(8.9855)
['limit'] tensor(8.6715)
['limitations'] tensor(8.3517)
['practical'] tensor(8.2851)
['a'] tensor(8.0979)
['problem'] tensor(7.9225)
['implementation'] tensor(7.8894)
['logic'] tensor(7.5877)
['size'] tensor(7.5874)
['rules'] tensor(7.4703)
['issues'] tensor(7.4602)
['computational'] tensor(7.3272)
['restrictions'] tensor(7.2864)
['theoretical'] tensor(7.1779)
['speed'] tensor(7.1758)
['efficiency'] tensor(7.1576)
['time'] tensor(7.0341)
['factors'] tensor(6.9810)
['amount'] tensor(6.9632)
['reduction'] tensor(6.9184)
['##s'] tensor(6.8955)


In [13]:
np.argmax(output[0][120])

tensor(13298)

In [14]:
tokenizer.ids_to_tokens[9586] 

'edgar'

In [15]:
from collections import Counter

# TODO
* add end token to model!

In [31]:
target = 120 #50
with torch.no_grad():
    model.eval()

    eval_loss_ans = 0
    for batch_i, eval_batch in enumerate(eval_dataloader_ans):
        eval_batch = tuple(t.to(device) for t in eval_batch)
        input_ids, input_mask, segment_ids, lm_label_ids, is_next = eval_batch
        if batch_i != target:
            continue
        if batch_i == target:
            #input_ids[0][121] = 103
            input_ids[0][118] = 103
            #input_ids[0][117] = 103
            print(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]))
        output, _ = model(input_ids, segment_ids, input_mask, None, None)
        if batch_i == target:
            break

['[CLS]', 'when', 'considering', 'computational', 'problems', ',', 'a', 'problem', 'instance', 'is', 'a', 'string', 'over', 'an', 'alphabet', '.', 'usually', ',', 'the', 'alphabet', 'is', 'taken', 'to', 'be', 'the', 'binary', 'alphabet', '(', 'i', '.', 'e', '.', ',', 'the', 'set', '{', '0', ',', '1', '}', ')', ',', 'and', 'thus', 'the', 'strings', 'are', 'bits', '##tri', '##ng', '##s', '.', 'as', 'in', 'a', 'real', '-', 'world', 'computer', ',', 'mathematical', 'objects', 'other', 'than', 'bits', '##tri', '##ng', '##s', 'must', 'be', 'suit', '##ably', 'encoded', '.', 'for', 'example', ',', 'integers', 'can', 'be', 'represented', 'in', 'binary', 'notation', ',', 'and', 'graphs', 'can', 'be', 'encoded', 'directly', 'via', 'their', 'ad', '##ja', '##cen', '##cy', 'matrices', ',', 'or', 'by', 'encoding', 'their', 'ad', '##ja', '##cen', '##cy', 'lists', 'in', 'binary', '.', '[SEP]', 'what', 'is', 'one', 'way', 'in', 'which', '[MASK]', 'can', 'be', 'encoded', '?', '[SEP]', '[PAD]', '[PAD]', '

In [32]:
print(tokenizer.convert_ids_to_tokens(np.argmax(output[0].data.numpy(), axis=1)))

['what', 'when', 'considering', 'computational', 'problems', ',', 'a', 'problem', 'instance', 'is', 'a', 'string', 'over', 'an', 'alphabet', '.', 'usually', ',', 'the', 'alphabet', 'is', 'taken', 'to', 'be', 'the', 'binary', 'alphabet', '(', 'i', '.', 'e', '.', ',', 'the', 'set', '{', '0', ',', '1', '}', ')', ',', 'and', 'thus', 'the', 'strings', 'are', 'bits', '##tri', '##ng', '##s', '.', 'as', 'in', 'a', 'real', '-', 'world', 'computer', ',', 'mathematical', 'objects', 'other', 'than', 'bits', '##tri', '##ng', '##s', 'must', 'be', 'suit', '##ably', 'encoded', '.', 'for', 'example', ',', 'integers', 'can', 'be', 'represented', 'in', 'binary', 'notation', ',', 'and', 'graphs', 'can', 'be', 'encoded', 'directly', 'via', 'their', 'ad', '##ja', '##cen', '##cy', 'matrices', ',', 'or', 'by', 'encoding', 'their', 'ad', '##ja', '##cen', '##cy', 'lists', 'in', 'binary', '.', 'graphs', 'what', 'is', 'one', 'way', 'in', 'which', '[MASK]', 'can', 'be', 'encoded', '?', 'graphs', 'graphs', 'graphs'

In [28]:
print(list(zip(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]), range(256))))


[('[CLS]', 0), ('when', 1), ('considering', 2), ('computational', 3), ('problems', 4), (',', 5), ('a', 6), ('problem', 7), ('instance', 8), ('is', 9), ('a', 10), ('string', 11), ('over', 12), ('an', 13), ('alphabet', 14), ('.', 15), ('usually', 16), (',', 17), ('the', 18), ('alphabet', 19), ('is', 20), ('taken', 21), ('to', 22), ('be', 23), ('the', 24), ('binary', 25), ('alphabet', 26), ('(', 27), ('i', 28), ('.', 29), ('e', 30), ('.', 31), (',', 32), ('the', 33), ('set', 34), ('{', 35), ('0', 36), (',', 37), ('1', 38), ('}', 39), (')', 40), (',', 41), ('and', 42), ('thus', 43), ('the', 44), ('strings', 45), ('are', 46), ('bits', 47), ('##tri', 48), ('##ng', 49), ('##s', 50), ('.', 51), ('as', 52), ('in', 53), ('a', 54), ('real', 55), ('-', 56), ('world', 57), ('computer', 58), (',', 59), ('mathematical', 60), ('objects', 61), ('other', 62), ('than', 63), ('bits', 64), ('##tri', 65), ('##ng', 66), ('##s', 67), ('must', 68), ('be', 69), ('suit', 70), ('##ably', 71), ('encoded', 72), ('.

In [35]:
target = 420 #50
with torch.no_grad():
    model.eval()

    eval_loss_ans = 0
    for batch_i, eval_batch in enumerate(eval_dataloader_ans):
        eval_batch = tuple(t.to(device) for t in eval_batch)
        input_ids, input_mask, segment_ids, lm_label_ids, is_next = eval_batch
        if batch_i != target:
            continue
        if batch_i == target:
            input_ids[0][45] = 103
            input_ids[0][46] = 103
            #input_ids[0][117] = 103
            print(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]))
        output, _ = model(input_ids, segment_ids, input_mask, None, None)
        if batch_i == target:
            break

['[CLS]', 'southern', 'california', 'is', 'home', 'to', 'many', 'major', 'business', 'districts', '.', 'central', 'business', 'districts', '(', 'cb', '##d', ')', 'include', 'downtown', 'los', 'angeles', ',', 'downtown', 'san', 'diego', ',', 'downtown', 'san', 'bernardino', ',', 'downtown', 'baker', '##sfield', ',', 'south', 'coast', 'metro', 'and', 'downtown', 'riverside', '.', '[SEP]', 'what', 'does', '[MASK]', '[MASK]', 'stand', 'for', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [36]:
print(tokenizer.convert_ids_to_tokens(np.argmax(output[0].data.numpy(), axis=1)))

['what', 'southern', 'california', 'is', 'home', 'to', 'many', 'major', 'business', 'districts', '.', 'central', 'business', 'districts', '(', 'cb', '##d', ')', 'include', 'downtown', 'los', 'angeles', ',', 'downtown', 'san', 'diego', ',', 'downtown', 'san', 'bernardino', ',', 'downtown', 'baker', '##sfield', ',', 'south', 'coast', 'metro', 'and', 'downtown', 'riverside', '.', 'cb', 'what', 'does', '[MASK]', '[MASK]', 'stand', 'for', '?', 'cb', '##d', '##d', '##d', '##d', '##d', '##d', '##d', 'streets', '##d', '##d', 'streets', '##d', '##d', '##d', '##d', '##d', '##d', '##d', 'districts', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', 'streets', 'streets', 'streets', 'area', 'area', 'streets', 'streets', 'area', 'streets', '##d', '##d', '##d', 'area', 'area', 'districts', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', '##d', 'streets', 'streets', 'streets', 'area', 'streets', 'streets', '##d', '##d'

In [34]:
print(list(zip(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]), range(256))))


[('[CLS]', 0), ('southern', 1), ('california', 2), ('is', 3), ('home', 4), ('to', 5), ('many', 6), ('major', 7), ('business', 8), ('districts', 9), ('.', 10), ('central', 11), ('business', 12), ('districts', 13), ('(', 14), ('cb', 15), ('##d', 16), (')', 17), ('include', 18), ('downtown', 19), ('los', 20), ('angeles', 21), (',', 22), ('downtown', 23), ('san', 24), ('diego', 25), (',', 26), ('downtown', 27), ('san', 28), ('bernardino', 29), (',', 30), ('downtown', 31), ('baker', 32), ('##sfield', 33), (',', 34), ('south', 35), ('coast', 36), ('metro', 37), ('and', 38), ('downtown', 39), ('riverside', 40), ('.', 41), ('[SEP]', 42), ('what', 43), ('does', 44), ('cb', 45), ('##d', 46), ('stand', 47), ('for', 48), ('?', 49), ('[SEP]', 50), ('[PAD]', 51), ('[PAD]', 52), ('[PAD]', 53), ('[PAD]', 54), ('[PAD]', 55), ('[PAD]', 56), ('[PAD]', 57), ('[PAD]', 58), ('[PAD]', 59), ('[PAD]', 60), ('[PAD]', 61), ('[PAD]', 62), ('[PAD]', 63), ('[PAD]', 64), ('[PAD]', 65), ('[PAD]', 66), ('[PAD]', 67), 

# TODO
* add end character to prediction
* eval only predicted masked terms
* normalize over reasonable options
    * handling multiple multi-token terms
    * the model may prefer one entity but getting the probability of a secondary or third entity may be hard because each position is conditioned individually 
* designate different types of mask for different decision points (entity, attr, q-type)
* implement automatic evaluation and exploration scripts

## Main story / pitch
* BERT is trained on huge amount of data and learns relationships / statistics of word correlations
* This model is conditional generation where you are outputing either a copy of what is in the paragraph or a paraphrase
* Most generated words should not be obvious by context but rather obvious by condition, and specific to context (ie. entities or attribtues)
* The model should be more about learning patterns and relations rather than term co-occurrence statistics

In [41]:
target = 720 #50
with torch.no_grad():
    model.eval()

    eval_loss_ans = 0
    for batch_i, eval_batch in enumerate(eval_dataloader_unans):
        eval_batch = tuple(t.to(device) for t in eval_batch)
        input_ids, input_mask, segment_ids, lm_label_ids, is_next = eval_batch
        if batch_i != target:
            continue
        if batch_i == target:
            input_ids[0][130] = 103
            input_ids[0][131] = 103
            #input_ids[0][117] = 103
            print(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]))
        output, _ = model(input_ids, segment_ids, input_mask, None, None)
        if batch_i == target:
            break

['[CLS]', 'montpellier', 'was', 'among', 'the', 'most', 'important', 'of', 'the', '66', '"', 'ville', '##s', 'de', 'sure', '##te', '"', 'that', 'the', 'edict', 'of', '159', '##8', 'granted', 'to', 'the', 'hug', '##uen', '##ots', '.', 'the', 'city', "'", 's', 'political', 'institutions', 'and', 'the', 'university', 'were', 'all', 'handed', 'over', 'to', 'the', 'hug', '##uen', '##ots', '.', 'tension', 'with', 'paris', 'led', 'to', 'a', 'siege', 'by', 'the', 'royal', 'army', 'in', '1622', '.', 'peace', 'terms', 'called', 'for', 'the', 'di', '##sman', '##tling', 'of', 'the', 'city', "'", 's', 'fortifications', '.', 'a', 'royal', 'citadel', 'was', 'built', 'and', 'the', 'university', 'and', 'consulate', 'were', 'taken', 'over', 'by', 'the', 'catholic', 'party', '.', 'even', 'before', 'the', 'edict', 'of', 'ale', '##s', '(', '1629', ')', ',', 'protestant', 'rule', 'was', 'dead', 'and', 'the', 'ville', 'de', 'sure', '##te', 'was', 'no', 'more', '.', '[', 'citation', 'needed', ']', '[SEP]', 'i

In [40]:
print(list(zip(tokenizer.convert_ids_to_tokens(input_ids.data.numpy()[0]), range(256))))


[('[CLS]', 0), ('montpellier', 1), ('was', 2), ('among', 3), ('the', 4), ('most', 5), ('important', 6), ('of', 7), ('the', 8), ('66', 9), ('"', 10), ('ville', 11), ('##s', 12), ('de', 13), ('sure', 14), ('##te', 15), ('"', 16), ('that', 17), ('the', 18), ('edict', 19), ('of', 20), ('159', 21), ('##8', 22), ('granted', 23), ('to', 24), ('the', 25), ('hug', 26), ('##uen', 27), ('##ots', 28), ('.', 29), ('the', 30), ('city', 31), ("'", 32), ('s', 33), ('political', 34), ('institutions', 35), ('and', 36), ('the', 37), ('university', 38), ('were', 39), ('all', 40), ('handed', 41), ('over', 42), ('to', 43), ('the', 44), ('hug', 45), ('##uen', 46), ('##ots', 47), ('.', 48), ('tension', 49), ('with', 50), ('paris', 51), ('led', 52), ('to', 53), ('a', 54), ('siege', 55), ('by', 56), ('the', 57), ('royal', 58), ('army', 59), ('in', 60), ('1622', 61), ('.', 62), ('peace', 63), ('terms', 64), ('called', 65), ('for', 66), ('the', 67), ('di', 68), ('##sman', 69), ('##tling', 70), ('of', 71), ('the',

In [42]:
print(tokenizer.convert_ids_to_tokens(np.argmax(output[0].data.numpy(), axis=1)))

['what', 'montpellier', 'was', 'among', 'the', 'most', 'important', 'of', 'the', '66', '"', 'ville', '##s', 'de', 'sure', '##te', '"', 'that', 'the', 'edict', 'of', '159', '##8', 'granted', 'to', 'the', 'hug', '##uen', '##ots', '.', 'the', 'city', "'", 's', 'political', 'institutions', 'and', 'the', 'university', 'were', 'all', 'handed', 'over', 'to', 'the', 'hug', '##uen', '##ots', '.', 'tension', 'with', 'paris', 'led', 'to', 'a', 'siege', 'by', 'the', 'royal', 'army', 'in', '1622', '.', 'peace', 'terms', 'called', 'for', 'the', 'di', '##sman', '##tling', 'of', 'the', 'city', "'", 's', 'fortifications', '.', 'a', 'royal', 'citadel', 'was', 'built', 'and', 'the', 'university', 'and', 'consulate', 'were', 'taken', 'over', 'by', 'the', 'catholic', 'party', '.', 'even', 'before', 'the', 'edict', 'of', 'ale', '##s', '(', '1629', ')', ',', 'protestant', 'rule', 'was', 'dead', 'and', 'the', 'ville', 'de', 'sure', '##te', 'was', 'no', 'more', '.', '[', 'citation', 'needed', ']', 'the', 'in',

In [43]:
c = Counter()
for i, o in enumerate(output[0][138]):
    c[i] = o
for x, val in c.most_common(25):
    print(tokenizer.convert_ids_to_tokens([x]), val)

['government'] tensor(12.8438)
['rule'] tensor(12.0960)
['monarchy'] tensor(11.6844)
['regime'] tensor(9.7085)
['democracy'] tensor(9.6858)
['city'] tensor(9.6177)
['system'] tensor(9.5993)
['power'] tensor(9.5922)
['institutions'] tensor(9.5776)
['university'] tensor(9.5670)
['society'] tensor(9.5314)
['walls'] tensor(9.3728)
['fortifications'] tensor(9.3701)
['structure'] tensor(9.3055)
['church'] tensor(9.2341)
['religion'] tensor(9.1682)
['order'] tensor(9.0762)
['resistance'] tensor(9.0731)
['castle'] tensor(8.7590)
['republic'] tensor(8.6874)
['catholicism'] tensor(8.6118)
['economy'] tensor(8.5853)
['institution'] tensor(8.5817)
['constitution'] tensor(8.5799)
['buildings'] tensor(8.4867)


In [45]:
c = Counter()
for i, o in enumerate(output[0][139]):
    c[i] = o
for x, val in c.most_common(25):
    print(tokenizer.convert_ids_to_tokens([x]), val)

['rule'] tensor(13.4089)
['government'] tensor(12.4013)
['monarchy'] tensor(11.1976)
['system'] tensor(10.8194)
['regime'] tensor(10.0926)
['power'] tensor(9.8284)
['structure'] tensor(9.3131)
['control'] tensor(9.1633)
['order'] tensor(9.0995)
['city'] tensor(9.0672)
['society'] tensor(8.9482)
['institutions'] tensor(8.9404)
['administration'] tensor(8.8783)
['walls'] tensor(8.6860)
['republic'] tensor(8.6619)
['democracy'] tensor(8.5542)
['resistance'] tensor(8.5440)
['situation'] tensor(8.5289)
['constitution'] tensor(8.5061)
['state'] tensor(8.4401)
['empire'] tensor(8.1867)
['buildings'] tensor(8.1807)
['institution'] tensor(8.1483)
['castle'] tensor(8.1160)
['laws'] tensor(8.0726)


In [ ]:
print(loss.item())

In [ ]:
print(tokenizer.convert_ids_to_tokens(context_ids.data.numpy()[0]))

In [ ]:
print(question_ids)
print(tokenizer.convert_ids_to_tokens(question_ids.data.numpy()[0]))

In [ ]:
o = output.data.numpy()
print(tokenizer.convert_ids_to_tokens(np.argmax(o[0], axis=1)))

In [ ]:
for i in range(10):
    print(np.argmax(o[0,i,:]), np.exp(np.max(o[0,i,:]))/ np.sum(np.exp(o[0,i,:])))

In [ ]:
from collections import Counter

In [ ]:
c = Counter()

In [ ]:
for i in range(o.shape[2]):
    c[i] = np.exp(o[0,7,i])/ np.sum(np.exp(o[0,7,:]))

In [ ]:
for idx, score in c.most_common()[:25]:
    print(tokenizer.convert_ids_to_tokens([idx]), score)

In [ ]:
c[2435] # normandy

In [ ]:
with torch.no_grad():
    model.eval()

    eval_loss_ans = 0
    for batch_i, eval_batch in enumerate(eval_dataloader_unans):
        eids = eval_batch[-1]
        eval_batch = tuple(t.to(device) for t in eval_batch[:-1])
        question_ids, question_mask, context_ids, context_mask, targets = eval_batch
        output, _ = model(context_ids, context_mask, question_ids, question_mask)
        loss = criterion(output.view(-1, len(tokenizer.vocab)), question_ids.view(-1))
        eval_loss_ans += loss.item()
        break
        if loss.item() > 0.01:
            print(batch_i, eval_loss_ans)
            break

In [ ]:
eval_loss_ans

In [ ]:
print(question_ids)
print(tokenizer.convert_ids_to_tokens(question_ids.data.numpy()[0]))

In [ ]:
o = output.data.numpy()
print(o)
print(tokenizer.convert_ids_to_tokens(np.argmax(o[0], axis=1)))

In [ ]:
for i in range(20):
    print(np.argmax(o[0,i,:]), np.exp(np.max(o[0,i,:]))/ np.sum(np.exp(o[0,i,:])))

# Interesting Examples

In [ ]:
from collections import Counter
def print_details(context_ids, question_ids, output, loss, tokenizer):
    print("Loss:", loss.item())
    print("CONTEXT")
    print(tokenizer.convert_ids_to_tokens(context_ids.data.numpy()[0]))
    print("~"*30)
    print("QUESTION")
    q_ids = [i for i in question_ids.data.numpy()[0] if i != 0]
    print(q_ids)
    q_toks = [tok for tok in tokenizer.convert_ids_to_tokens(question_ids.data.numpy()[0]) if tok != '[PAD]']
    print(q_toks)
    print("~"*30)
    print("OUTPUT")
    o = output.data.numpy()
    out_ids = [i for i in np.argmax(o[0], axis=1) if i != 0]
    out_toks = [tok for tok in tokenizer.convert_ids_to_tokens(np.argmax(o[0], axis=1)) if tok != '[PAD]']
    scores = [(np.argmax(o[0,i,:]), np.exp(np.max(o[0,i,:]))/ np.sum(np.exp(o[0,i,:]))) for i in range(len(out_toks))]
    print(out_toks)
    print(list(zip(out_toks, scores)))
    print("~"*30)
    print("TOP K FOR INCORRECT TERMS:")
    for tok_i, (tar, out) in enumerate(zip(q_ids[1:], out_ids)):
        if tar != out:
            print ("Output", out_toks[tok_i], "instead of ", q_toks[tok_i+1])
            c = Counter()
            for i in range(o.shape[2]):
                c[i] = np.exp(o[0,tok_i,i])/ np.sum(np.exp(o[0,tok_i,:]))
            for idx, score in c.most_common()[:10]:
                print("- \t",tokenizer.convert_ids_to_tokens([idx]), score)
    print("#"*30)

In [ ]:
explore = {eval_dataloader_ans: [0,1], eval_dataloader_unans: [0, 15]}

with torch.no_grad():
    model.eval()

    eval_loss_ans = 0
    for dataloader in explore.keys():
        for batch_i, eval_batch in enumerate(dataloader):
            eids = eval_batch[-1]
            eval_batch = tuple(t.to(device) for t in eval_batch[:-1])
            question_ids, question_mask, context_ids, context_mask, targets = eval_batch
            output, _ = model(context_ids, context_mask, question_ids, question_mask)
            loss = criterion(output.view(-1, len(tokenizer.vocab)), targets.view(-1))
            eval_loss_ans += loss.item()
            if batch_i in explore[dataloader]:
                if dataloader == eval_dataloader_ans:
                    print("Answerable:",batch_i)
                else:
                    print("Unanswerable:",batch_i)
                print_details(context_ids, question_ids, output, loss, tokenizer)
            if batch_i > max(explore[dataloader]):
                break


# Notes
* LM was suprisingly good at guessing question type and general structure of the question. It may be because the BERT representation is leaky in terms of representation  
    - may be interesting result of its own ... Q: How much does the feature of each BERT word contain information about the surrounding BERT words?  
    - this is probably why the model is pretty good at guessing Q type from the [CLS] token  
* the Sentence repr is not enough represent specific entities in the text  
    - gets confused between normandy and france and mongolia and normans   
    - this might be because I don't train the sentence repr which is probably a mistake and we should retrain this one with trained sentence repr and forward masking in the question repr
    - Maybe we can do an attention over entities? 
    - There is a bias towards more common entities right now (ie. john and paris instead of rollo and normandy)
    - the fact that stuff like Rollo and dates were predicted well means there is leakiness in the representation
* rank may be more important than loss since sometimes the #1 option has very high prob but number 2 is really good (eg. arrive vs begin U15)